In [17]:
from neuron import h, gui
import bokeh as bk
import bokeh.plotting as bkp
import numpy as np
from bokeh.palettes import Dark2_5 as palette
import itertools 
import ipywidgets as widgets
from branched_cable import branched_cable, run_current_clamp

colors = itertools.cycle(palette)  
bkp.output_notebook()

Loading BokehJS ...

In [18]:
def plot_single_compartment_IClamp(diam=10,Rm=2800,cm=1):
    
    #t, vArr, I = run_single_compartment_IClamp(diam,Rm,cm)
    cell = branched_cable()
    cell.make_cable(diam,diam,nseg = 1,Rm=Rm,cm=cm)
    cell.add_IClamp('cable',.5,.001,25,delay = 20)
    cell.recording_vecs([['cable',.5,'v']])

    I = np.linspace(-.2,.2,num=7)
    t, results = run_current_clamp(cell,'iclamp',I,tstop=80)
    
    vArr =  results['cable:0.50:v']
    
    RC= bkp.figure(plot_width=400,plot_height=400)
    RC.xaxis.axis_label="time"
    RC.yaxis.axis_label="V (mV)"
    colors = itertools.cycle(palette)  
    
    
    for i, color in zip(range(vArr.shape[1]),colors):
        RC.line(t,vArr[:,i],color=color) #,legend=str(I[i]))
    #RC.legend.location="top_right"
    
    #find steady state voltage
    VI = bkp.figure(plot_height=400,plot_width=400)
    VI.xaxis.axis_label="I (mA)"
    VI.yaxis.axis_label="V (mV)"
    ss_ind = np.where(t>43)[0][0]
    V_ss = vArr[ss_ind,:]
    VI.line(I,V_ss)
    
    
    # time constant
    vArr = vArr +65
    V_half = (1-.37)*(V_ss+65.)
    tau = np.zeros([7,])
    for col in range(I.size):
        tau[col] = np.where(vArr[:,i]>V_half[i])[0][0]/t.size*80. -20
        
    T = bkp.figure(plot_height=400,plot_width=400)
    T.xaxis.axis_label="I (mA)"
    T.yaxis.axis_label= "tau"
    T.line(I,tau)
    
    bkp.show(bk.layouts.layout([[RC,VI],[T]]))
    
             




In [19]:
widgets.interact_manual(plot_single_compartment_IClamp,
                 diam=widgets.FloatSlider(min=.1,max=100,continuous_update=False),
                 Rm=widgets.FloatSlider(min=100,max=5000,continuous_update=False),
                 cm = widgets.FloatSlider(min=.1,max=10,continous_update=False))

A Jupyter Widget

<function __main__.plot_single_compartment_IClamp>

Insert JPeg of circuit diagram

Insert Latex of current equation

# Steady State Properties
## V-I curve analysis

In [20]:
I = np.linspace(-.2,.2,num=7)
def get_VI(diam,Rm,cm):
    cell = branched_cable()  
    cell.make_cable(diam,diam,nseg = 1,Rm=Rm,cm=cm)
    cell.add_IClamp('cable',.5,.001,25,delay = 20)
    cell.recording_vecs([['cable',.5,'v']])

    t, results = run_current_clamp(cell,'iclamp',I,tstop=80)
    
    vArr =  results['cable:0.50:v']
    ss_ind = np.where(t>43)[0][0]
    V_ss = vArr[ss_ind,:]
    
    slope = np.diff(V_ss).mean()
    return V_ss,slope

In [21]:
# V-I slope vs diam
diams = np.linspace(10,100,num=50)
V_ss = np.zeros([I.shape[0],50])
slope = np.zeros([50,])
for i, diam in enumerate(list(diams)):  
    V_ss[:,i], slope[i] = get_VI(diam,2800,1)
    
VI = bkp.figure(plot_height=400,plot_width=400)
for i, color in zip(range(0,50,5),colors):
    VI.line(I,V_ss[:,i],color=color)
VI.xaxis.axis_label="I (mA)"
VI.yaxis.axis_label="mV"
    
S = bkp.figure(plot_height=400,plot_width=400)
S.yaxis.axis_label="Input Resistance"
S.xaxis.axis_label="diameter"
S.line(diams,slope)
bkp.show(bk.layouts.layout([[VI,S]]))



In [22]:
# V-I slope vs Rm
Rms = np.logspace(2,5,num=50)
V_ss = np.zeros([I.shape[0],50])
slope = np.zeros([50,])
for i, Rm in enumerate(list(Rms)):  
    V_ss[:,i], slope[i] = get_VI(20,Rm,1)
    
VI = bkp.figure(plot_height=400,plot_width=400)
for i, color in zip(range(0,50,5),colors):
    VI.line(I,V_ss[:,i],color=color)
VI.xaxis.axis_label="I (mA)"
VI.yaxis.axis_label="mV"
    
S = bkp.figure(plot_height=400,plot_width=400)
S.yaxis.axis_label="Input Resistance"
S.xaxis.axis_label="Rm (MOhm/cm^2)"
S.line(Rms,slope)
bkp.show(bk.layouts.layout([[VI,S]]))

# V-i slope vs Cm

In [23]:
# V-i slope vs Cm
cms = np.linspace(.01,5,num=50)
V_ss = np.zeros([I.shape[0],50])
slope = np.zeros([50,])
for i, cm in enumerate(list(cms)):  
    V_ss[:,i], slope[i] = get_VI(20,2800,cm)
    
VI = bkp.figure(plot_height=400,plot_width=400)
for i, color in zip(range(0,50,5),colors):
    #if i%5==0:
    VI.line(I,V_ss[:,i],color=color)
VI.xaxis.axis_label="I (mA)"
VI.yaxis.axis_label="mV"
    
S = bkp.figure(plot_height=400,plot_width=400)
S.yaxis.axis_label="Input Resistance"
S.xaxis.axis_label="Cm (pF/cm^2)"
S.line(cms,slope)
bkp.show(bk.layouts.layout([[VI,S]]))



# Dynamics
## V_1/2 analysis


In [11]:
I = [.2]
def get_tau(diam,Rm,cm):
    cell = branched_cable()
    cell.make_cable(diam,diam,nseg = 1,Rm=Rm,cm=cm)
    cell.add_IClamp('cable',.5,.001,500,delay = 20)
    cell.recording_vecs([['cable',.5,'v']])

    t, results = run_current_clamp(cell,'iclamp',I,tstop=700)
    vArr =  results['cable:0.50:v']
    
    vZero = vArr+65
    ss_ind = np.where(t>520)[0][0]
    V_ss = vZero[ss_ind]
    V_half = (1-.37)*V_ss
    tau = np.where(vZero>V_half)[0][0]/t.size*80. -20
    
    return vArr, tau

In [12]:
# Diam

diams = np.linspace(10,100,num=50)
tau = np.zeros([50,])
for i, diam in enumerate(list(diams)):  
    tmp, tau[i] = get_tau(diam,2800,1)
    
    
S = bkp.figure(plot_height=400,plot_width=400)
S.yaxis.axis_label="tau"
S.xaxis.axis_label="diameter"
S.line(diams,tau)
bkp.show(S)


In [13]:
# Vary Rm

Rms = np.logspace(2,5,num=50)
tau = np.zeros([50,])
for i, Rm in enumerate(list(Rms)):  
    tmp, tau[i] = get_tau(20,Rm,1)
    
    
S = bkp.figure(plot_height=400,plot_width=400)
S.yaxis.axis_label="tau"
S.xaxis.axis_label="Rm"
S.line(Rms,tau)
bkp.show(S)

In [14]:
# Vary Cm
cms = np.linspace(.01,5,num=50)
tau = np.zeros([50,])
for i, cm in enumerate(list(cms)):  
    tmp, tau[i] = get_tau(20,2800,cm)
    
    
S = bkp.figure(plot_height=400,plot_width=400)
S.yaxis.axis_label="tau"
S.xaxis.axis_label="Cm"
S.line(cms,tau)
bkp.show(S)